# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**: Elliot Walsh

**ID**: ejw224, 5101573

> **Due Date**
>
> Friday, 10/13/23, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [1]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Desktop/fall 2023/cee 4750/lab 3/lab03-ejw224`


In [2]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver
using Plots # plotting

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

For an example of using `JuMP.jl` to solve linear programs, see [the
relevant tutorial on the class
website](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html).

Free free to delete some of the illustrative cells and code blocks in
your notebook as you go through and solve the lab problems…this might
help reduce some potential confusion while grading about what your
answer is.

## Introduction

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
\$0.12 per bf to purchase and fir costs \$0.08 per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board. This resource
allocation problem is diagrammed in
<a href="#fig-schematic" class="quarto-xref">Figure 1</a>.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.


## Problems (10 points)

### Problem 1: Problem Formulation (5 points)

In this problem, you will go through the steps of formulating a linear
program for this problem.

#### Problem 1.1 (1 point)

What are your decision variables? Clearly define your notation,
including what variables you are using, what they mean, and what their
units are.

The decision variables are:

**p1**: amount of grade 1 plywood produced (1000s of bf)

**p2**: amount of grade 2 plywood produced (1000s of bf)

**p3**: amount of grade 3 plywood produced (1000s of bf)

#### Problem 1.2 (1 point)

Derive your objective function. Support your function with
justifications and/or equations as necessary. You will not receive
credit just for the function alone.

Goal is to maximize profit (as revenues-costs). Revenue is the value of the plywood produced, and cost is the cost to buy the unprocessed wood. s and f are the prices of spruce and fir in $/(1000 bf)

Revenue: sum of values in table: $$R=400p_1+520p_2+700p_3 $$

Cost: sum of materials given in table at given prices: $$C=(0.5s+1.5f)p_1+(s+2f)p_2+(1.5s+2f)p_3$$

Profit: revenue-cost: $$P=R-C=(400p_1+520p_2+700p_3)-((0.5s+1.5f)p_1+(s+2f)p_2+(1.5s+2f)p_3)$$

#### Problem 1.3 (2 point)

Derive any needed constraints. Support your function with justifications
and/or equations as necessary. You will not receive credit just for the
final constraints alone.

Constraints are based on the availability of spruce and fir, as given in the introduction, and the fact that s, f, p1, p2, and p3 must all be nonnegative: 
$$0 \leq 0.5p_1+p_2+1.5p_3	\leq 320 $$
$$0 \leq 1.5p_1+2p_2+2p_3 \leq 720$$
$$0 \leq p_1$$
$$0 \leq p_2$$
$$0 \leq p_3$$


#### Problem 1.4 (1 point)

Put this optimization problem in mathematical programming form. For an
example of the syntax for this, see lines 82–91
[here](https://github.com/vsrikrish/environmental-systems-analysis/blob/Fall23/tutorials/julia-jump.qmd).

\begin{equation}
\begin{aligned}
& \max_{p_1, p_2, p_3} & ((400p_1+520p_2+700p_3))-((0.5s+1.5f)p_1+(s+2f)p_2+(1.5s+2f)p_3)\\
&\text{subject to} & \\
& & 0 \leq 0.5p_1+p_2+1.5p_3	\leq 320\\
& & 0 \leq 1.5p_1+2p_2+2p_3 \leq 720\\
& & 0 \leq p_1\\
& & 0 \leq p_2\\
& & 0 \leq p_3 
\end{aligned}
\end{equation}

### Problem 2: Find the Solution (5 points)

#### Problem 2.1 (2 points)

Code your linear program using `JuMP`. Feel free to consult [the
website’s `JuMP`
tutorial](https://viveks.me/environmental-systems-analysis/tutorials/julia-jump.html)
for syntax help. The keys:

In [3]:
s=.12*1000;
f=.08*1000;
model_22 = Model(HiGHS.Optimizer)
@variable(model_22, 0 <= p1)
@variable(model_22, 0 <= p2)
@variable(model_22, 0 <= p3)
@objective(model_22, Max, 400p1+520p2+700p3-((0.5s+1.5f)p1+(s+2f)p2+(1.5s+2f)p3))
@constraint(model_22, spruceMin, 0.5*p1+p2+1.5*p3 >= 0)
@constraint(model_22, spruceMax, 0.5*p1+p2+1.5*p3 <= 320)
@constraint(model_22, firMin, 1.5*p1+2*p2+2*p3 >= 0)
@constraint(model_22, firMax, 1.5*p1+2*p2+2*p3 <= 720)
print(model_22)

Max 220 p1 + 240 p2 + 360 p3
Subject to
 

spruceMin : 0.5 p1 + p2 + 1.5 p3 ≥ 0
 firMin : 1.5 p1 + 2 p2 + 2 p3 ≥ 0
 spruceMax : 0.5 p1 + p2 + 1.5 p3 ≤ 320
 firMax : 1.5 p1 + 2 p2 + 2 p3 ≤ 720
 p1 ≥ 0
 p2 ≥ 0
 p3 ≥ 0


#### Problem 2.2 (1 points)

Find the solution to your program and find the optimal values of the
decision variables. Once you’ve defined your model, you can find the
solution with \`optimize!():

In [4]:

optimize!(model_22)

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
2 rows, 3 cols, 6 nonzeros
2 rows, 3 cols, 6 nonzeros
Presolve : Reductions: rows 2(-2); columns 3(-0); elements 6(-6)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.1999929744e+02 Ph1: 2(8.5); Du: 3(819.999) 0s
          2    -1.1200000000e+05 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.00


In [5]:
print("p1 = ", value.(p1), ", p2 = ", value.(p2), ", p3 = ", value.(p3))


p1 = 352.00000000000006, p2 = 0.0, p3 = 95.99999999999999

#### Problem 2.3 (1 point)

How would your profit change if you could buy 1,000 additional bf of
spruce? You can answer this by getting the shadow price of a particular
variable with:

In [6]:
print("the profit would increase by \$", shadow_price(spruceMax))

the profit would increase by $80.0

#### Problem 2.4 (1 point)

Would you prefer to have 2,000 additional bf of spruce or 1,000
additional bf of fir?

In [7]:
shadow_price(firMax)

120.0

I would rather have the additional 2000 bf of spruce, because the shadow price of the maximum spruce constraint is greater than half that of the maximum fir constraint

## References

Put any consulted sources here, including classmates you worked with/who
helped you.